In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
import numpy as np
import numpy as np
import torch
from pysmiles import read_smiles
import networkx as nx
import dgllife
from rdkit import Chem
import random
import dgl.nn.pytorch as dglnn
import torch.nn as nn
import dgl
from dgl import DGLGraph
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import pandas
import csv
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from torch.distributions import Categorical
import matplotlib.pyplot as plt
import functools
#%run MoleculeGenerator2.ipynb
%run Discrim.ipynb
%run ChemEnv.ipynb
%run MolUtils.ipynb
%run Tests.ipynb



In [ ]:
batch = dgl.load_graphs('./graph_decomp/chunk_0')

In [ ]:
MolFromGraphsFULL(batch[0][10])

In [ ]:
drugs = pandas.read_csv('SmallDrug.csv',error_bad_lines=False,delimiter=';')
smiles_values = drugs['Smiles'].values


In [ ]:
len(smiles_values)

In [ ]:
def data_base_generator(smiles,chunk_size, path):
    '''function for generating and storing decomposition trajectories'''
    i = chunk_size 
    j=0
    while i < len(smiles):
        graphs = []
        last_action_nodes = []
        actions = []
        smiles_chunk = smiles[i-chunk_size: i]
        
        for smile in smiles_chunk:
            if mol_checker(smile,'SMILES'):
                graph = smiles_to_graph([smile])[0]
                graphs_single,last_action_nodes_single, actions_single = full_graph_decomp(graph)
                
                graphs.extend(graphs_single)
                last_action_nodes.extend(last_action_nodes_single)
                actions.extend(actions_single)
        print(torch.tensor(last_action_nodes).shape)
        print(torch.tensor(actions).shape)
        
        graph_labels = {"last_action" : torch.tensor(last_action_nodes), 
                        "actions" : torch.tensor(actions)}
        
    
        dgl.save_graphs('./graph_decomp/chunk_' + str(j),graphs, graph_labels)
        j+=1
        i+= chunk_size
        print(i)
    
    
    

In [ ]:
def DataBaseGeneration(smiles,chunk_size):
    i = chunk_size
    j = 0
    atom_types = ['N','C','O','S','F','Cl','Na','P','Br','Si','B','Se','K']

    while i < len(smiles):
        state_list = []
        action_list = []
        smiles_chunk = smiles[i-chunk_size: i]
        
        for smile in smiles_chunk:
            try:
                mol = Chem.MolFromSmiles(smile)
            except:
                continue
            if all([atom.GetSymbol() in atom_types for atom in mol.GetAtoms()]) and all([bond.GetBondType() != Chem.BondType.TRIPLE for bond in mol.GetBonds()]):
                states,actions = FullMolDecomp(mol)
                state_list.extend(states)
                action_list.extend(actions)
                
                
        zipped = list(zip(state_list,action_list))
        random.shuffle(zipped)
        state_list_shuffle,action_list_shuffle = zip(*zipped)
        graph_list, last_action_node_list, last_atom_feat_list = zip(*state_list_shuffle)
        
        graph_labels = {"last_action" : torch.tensor(last_action_node_list),
                        "last_atom_feats" : torch.cat(last_atom_feat_list,dim=0),
                        "actions" : torch.tensor(action_list_shuffle)}
        #print(llen(graphen(graph_labels["last_action"],len(graph_labels["last_atom_feats"]),len(graph_labels["actions"])))
        print(type(graph_list))
        dgl.save_graphs('./graph_decomp/chunk_' + str(j),list(graph_list), graph_labels)
        j+=1
        i+= chunk_size
        print(i)
        
        
        

In [ ]:
def DataBaseGenerationBIGSHUFFLE(smiles,chunk_size):
    i = 0
    j = 0
    atom_types = ['N','C','O','S','F','Cl','Na','P','Br','Si','B','Se','K']
    
    state_list = []
    action_list = []

    for smile in smiles:
        try:
            mol = Chem.MolFromSmiles(smile)
        except:
            continue
        if all([atom.GetSymbol() in atom_types for atom in mol.GetAtoms()]) and all([bond.GetBondType() != Chem.BondType.TRIPLE for bond in mol.GetBonds()]):
            states,actions = FullMolDecomp(mol)
            state_list.extend(states)
            action_list.extend(actions)
            
            
            
    zipped = list(zip(state_list,action_list))
    random.shuffle(zipped)
    state_list_shuffle,action_list_shuffle = zip(*zipped)
    graph_list, last_action_node_list, last_atom_feat_list = zip(*state_list_shuffle)

    while i < len(state_list) - chunk_size:
        _slice = slice(i,i+chunk_size)
        
        last_action_node_list_slice = last_action_node_list[_slice]
        last_atom_feat_list_slice = last_atom_feat_list[_slice]
        
        graph_list_slice = graph_list[_slice]
        
        action_list_slice = action_list_shuffle[_slice]
        
        graph_labels = {"last_action" : torch.tensor(last_action_node_list_slice),
                        "last_atom_feats" : torch.cat(last_atom_feat_list_slice,dim=0),
                        "actions" : torch.tensor(action_list_slice)}
        
        dgl.save_graphs('./graph_decomp/chunk_' + str(j),list(graph_list_slice), graph_labels)
        j+=1
        i+= chunk_size
        print(i)
        
        
        

In [ ]:
def DataBaseGenerationONEFILE(smiles):
    i = 0
    j = 0
    atom_types = ['N','C','O','S','F','Cl','Na','P','Br','Si','B','Se','K']
    
    state_list = []
    action_list = []

    for i,smile in enumerate(smiles):
        if i%1000 == 0:
            print(i)
        try:
            mol = Chem.MolFromSmiles(smile)
        except:
            continue
        if all([atom.GetSymbol() in atom_types for atom in mol.GetAtoms()]) and all([bond.GetBondType() != Chem.BondType.TRIPLE for bond in mol.GetBonds()]):
            states,actions = FullMolDecomp(mol)
#             for state in states:
#                 print(state)
            state_list.extend(states)
            action_list.extend(actions)
            
            
            
#     zipped = list(zip(state_list,action_list))
#     random.shuffle(zipped)
#     state_list_shuffle,action_list_shuffle = zip(*zipped)
#     graph_list, last_action_node_list, last_atom_feat_list = zip(*state_list_shuffle)
    
#     print(torch.tensor(last_action_node_list).shape,torch.stack(last_atom_feat_list,dim=0).shape)
    graph_list, last_action_node_list, last_atom_feat_list = zip(*state_list)
    
    graph_labels = {"last_action" : torch.tensor(last_action_node_list),
                    "last_atom_feats" : torch.cat(last_atom_feat_list,dim=0),
                    "actions" : torch.tensor(action_list)}

    dgl.save_graphs('./graph_decomp/full_chunka',list(graph_list), graph_labels)

        
        
        

In [ ]:
DataBaseGenerationONEFILE(smiles_values)

In [ ]:
graphs = dgl.load_graphs('./graph_decomp/full_chunka')

In [ ]:
graphs, g_dict = graphs

In [ ]:
i = 3
print(g_dict['actions'][i], g_dict['last_action'][i])
MolFromGraphsFULL(graphs[i])

In [ ]:
def FullMolDecomp(mol, return_mol = False):
    
    
    
    graph = selfLoop(mol_to_graph_full(mol))
    last_action_node = torch.zeros((1,1)).to(device)
    
    #states = [graph, last_action_node, torch.unsqueeze(graph.ndata['atomic'][-1],dim=0)]
    obs = (graph, last_action_node, torch.unsqueeze(graph.ndata['atomic'][-1],dim=0))
    states = [obs]
    actions = [torch.zeros(1)]
    mol_list = [mol]
    
    
    i = 1
    success = True
    while success:
        mol, obs, action, success = MolDecompStep(mol)
        if success:
            states.append(obs)
            actions.append(action)
            mol_list.append(mol)
    if return_mol:
        return mol_list, states, actions
    
    return states,actions
        
        
        
    

In [ ]:
def MolDecompStep(mol):
    '''
    input: RDKit Mol 
    Output: (RDKit Mol, last action node, node features) 
    
    Priority:
    
    1. Remove Lone Node/Aromatic ring
    2. Disconnect Aromatic Ring
    3. Disconnect Atom
    4. Break Cycle
    '''
    mol_copy = permute_mol(mol, lambda x: x) 
    Chem.SanitizeMol(mol_copy)
    success = False
    
    if not success:
        mol, obs, action, success = remove_node(mol_copy)
    
    if not success:
        mol, obs, action, success = remove_aro_ring(mol_copy)
    
        
    if not success:
        mol, obs, action, success = remove_ring_edge(mol_copy)
        
    if not success:
        mol, obs, action, success = remove_node_edge(mol_copy)    
              
    if not success:
        mol, obs, action, success = break_cycle(mol_copy)
      
        
    if success and mol.GetNumAtoms()>1:
        return mol, obs, action, success 
    
    return None, None, None, False
    
    
    
    
    
    
    

In [ ]:
def _remove_edge(mol, atom1_idx, atom2_idx,len_node_actions=16):
    if atom1_idx == mol.GetNumAtoms()-1:
        atom1_idx, atom2_idx = atom2_idx, atom1_idx
    new_mol = permuteAtomToEnd(mol,atom2_idx)
    b = new_mol.GetBondBetweenAtoms(new_mol.GetNumAtoms()-1,atom1_idx)
    bt = b.GetBondType()
    
    if bt == 'SINGLE':
        bt_int = 0
    else:
        bt_int = 1
        
    new_mol.RemoveBond(new_mol.GetNumAtoms()-1,atom1_idx)
    action =  2*atom1_idx + len_node_actions + bt_int
    
    return new_mol, action
    


In [ ]:
def remove_node(mol):
    
    atom_list = ['N','C','O','S','F','Cl','Na','P','Br','Si','B','Se','K', 'Aro']    
    for atom in mol.GetAtoms():
        if atom.GetDegree() == 0:
            atom_idx = atom.GetIdx()
            atom_symbol = atom.GetSymbol() 

            action = atom_list.index(atom_symbol) + 1
            mol.RemoveAtom(atom_idx)
            
            
            try:
                Chem.SanitizeMol(mol)
                graph = mol_to_graph_full(mol)
                last_action_node = torch.zeros((1,1)).to(device)
                obs = graph, last_action_node, torch.unsqueeze(graph.ndata['atomic'][-1], dim = 0)                
                return mol, obs, action, True
            except Chem.rdchem.KekulizeException:
                return (None,None,None), -1, False 
    
    
    return None, (None,None,None), -1, False
        
        
        

In [ ]:
#decide on aromatic ring ordering 
#ORDERING WILL BE OFF UNTILL DECIDING ON HOW MANY RINGS TO ALLOW
def remove_aro_ring(mol):
    aro_dict_list = [{'C': 6}, {'N': 1, 'C': 5}, {'N': 1, 'C': 4}]
    aro_list = ['Benz', 'Pyri', 'Pyrr']
    cycles = list(mol.GetRingInfo().AtomRings())
    
    for cycle in cycles:
        if all([mol.GetAtomWithIdx(atom_id).GetIsAromatic() for atom_id in cycle]) and all([mol.GetAtomWithIdx(atom_id).GetDegree() == 2 for atom_id in cycle]):
            aro_cycle = cycle
            atom_types_dict = (Counter([mol.GetAtomWithIdx(atom_id).GetSymbol() for atom_id in cycle]))
            
            try:
                aro_idx = aro_dict_list.index(atom_types_dict)
            except:
                continue
            aro_idx_name = aro_list[aro_idx]
    
            for atom_idx in sorted(aro_cycle, reverse = True):
                mol.RemoveAtom(atom_idx)
            action = 1 + 13 + aro_idx
            
            if mol.GetNumAtoms() == 0:
                    return None, (None,None,None), -1, False 
            try:
                Chem.SanitizeMol(mol)
                graph = mol_to_graph_full(mol)
                last_action_node = torch.zeros((1,1)).to(device)
                obs = graph, last_action_node, torch.unsqueeze(graph.ndata['atomic'][-1], dim = 0)   
                
                return mol, obs, action, True
            except Chem.rdchem.KekulizeException:
                return None, (None,None,None), -1, False 
        
    
    return None, (None,None,None), -1, False 
            
                
    

In [ ]:
#edge_type 
def remove_node_edge(mol):    
    for atom in mol.GetAtoms():
        if atom.GetDegree() == 1:
            atom_idx = atom.GetIdx()
            b = list(atom.GetBonds())[0]
            other_atom_idx = b.GetOtherAtom(atom).GetIdx()
            new_mol, action = _remove_edge(mol,other_atom_idx,atom_idx)
            
            try:
                Chem.SanitizeMol(new_mol)
                graph = mol_to_graph_full(new_mol)
                last_action_node = torch.ones((1,1)).to(device)
                obs = graph, last_action_node, torch.unsqueeze(graph.ndata['atomic'][-1], dim = 0)                
                return new_mol, obs, action, True
            except Chem.rdchem.KekulizeException:
                return None, (None,None,None), -1, False 
            
                
    
    return None, (None,None,None), -1, False

In [ ]:
def remove_ring_edge(mol):
    cycles = list(mol.GetRingInfo().AtomRings())
    for cycle in cycles:
        degrees = [mol.GetAtomWithIdx(cycle_idx).GetDegree() for cycle_idx in cycle] 
        is_aromatic = [mol.GetAtomWithIdx(cycle_idx).GetIsAromatic() for cycle_idx in cycle] 
        if sum(degrees) == (len(degrees) * 2) + 1 and all(is_aromatic):
            idx_in_cycle = degrees.index(3)
            
            atom_idx = cycle[idx_in_cycle]
            atom = mol.GetAtomWithIdx(atom_idx)
            b_list = list(atom.GetBonds())
            
            
            for b in b_list:
                bt = b.GetBondType()
                if bt != Chem.rdchem.BondType.AROMATIC:
                    break
            else:
                continue
                
                
            other_atom_idx = b.GetOtherAtom(atom).GetIdx()
            new_mol, action = _remove_edge(mol,atom_idx,other_atom_idx)
            
            
            
            try:
                Chem.SanitizeMol(new_mol)
                graph = mol_to_graph_full(new_mol)
                last_action_node = torch.ones((1,1)).to(device)
                obs = graph, last_action_node, torch.unsqueeze(graph.ndata['atomic'][-1], dim = 0)                
                return new_mol, obs, action, True
            except Chem.rdchem.KekulizeException:
                return None, (None,None,None), -1, False 
                
        
    
    return None, (None,None,None), -1, False 
            
            
            
            

In [ ]:
def break_cycle(mol):
        for atom_i in range(mol.GetNumAtoms()):
            if mol.GetAtomWithIdx(atom_i).GetIsAromatic():
                continue
            for atom_j in range(atom_i+1, mol.GetNumAtoms()):
                if mol.GetAtomWithIdx(atom_j).GetIsAromatic():
                    continue
                bond = mol.GetBondBetweenAtoms(atom_i,atom_j)
                if bond == None:
                    continue
                
                bond_type_int = bondTypeInt(bond.GetBondType())
                mol_copy = permute_mol(mol,lambda x: x)
                mol_copy.RemoveBond(atom_i,atom_j)
                if not nx.is_connected(mol_to_graph(mol_copy).to_networkx().to_undirected()):
                    continue
                new_mol, action = _remove_edge(mol,atom_i,atom_j)
                graph = mol_to_graph_full(new_mol)
                last_action_node = torch.ones((1,1)).to(device)
                obs = graph, last_action_node, torch.unsqueeze(graph.ndata['atomic'][-1], dim = 0)
                Chem.SanitizeMol(new_mol)
                return new_mol, obs, action, True
        else:
            return None, (None,None,None), -1, False 
            
                    
                    

In [ ]:
graph_l, dic = dgl.load_graphs('./graph_decomp/full_chunk0')

In [ ]:
#check value 23

In [ ]:
test_mol = Chem.MolFromSmiles(smiles_values[19])
moldecomp = FullMolDecomp(test_mol,return_mol=True)
test_mol

In [ ]:
i = 5
print(moldecomp[2][i])
moldecomp[0][i]

In [ ]:
dmol,obs,action,val = MolDecompStep(dmol)
print(action)
dmol


In [ ]:
i = 4
print(dic['actions'][i])
mol_x = MolFromGraphsFULL(graph_l[i])
mol_x

In [ ]:
cvt.Clear
cvt.AssignMol(mol_x)
cvt.Step(int(dic['actions'][i]+4))
cvt.GetMol()

In [ ]:
n_mol, obs, action, value = remove_ring_edge(mol)
print(action)

In [ ]:
cvt.Clear
cvt.AssignMol(n_mol)
print(action)
cvt.Step(action)
cvt.GetMol()

In [ ]:
wgraph, wdict = dgl.load_graphs('./graph_decomp/full_chunkx0')

In [ ]:
i = 1
print(wdict['actions'][i])
MolFromGraphsFULL(wgraph[i])

In [ ]:
test_mol = Chem.MolFromSmiles(smiles_values[19])
moldecomp = FullMolDecomp(test_mol,return_mol=True)
test_mol

In [ ]:
j = 5

print(int(moldecomp[2][j]))
moldecomp[0][j]